#Não tem o arquivo de referência <br><br>

https://github.com/ivanovitchm/deeplearning/blob/main/weeks_02_03/Notebooks/Week%2002%20Task%2002%20-%20Introduction%20to%20TF.ipynb

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time

In [ ]:
def load_dataset():
    # load the train data
    train_dataset = h5py.File('train_catvnoncat.h5', "r")

    # your train set features
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) 

    # your train set labels
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) 

    
    # load the test data
    test_dataset = h5py.File('test_catvnoncat.h5', "r")

    # your test set features
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) 

    # your test set labels  
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) 

    # the list of classes
    classes = np.array(test_dataset["list_classes"][:]) 

    # reshape the test data
    train_set_y_orig = train_set_y_orig.reshape((train_set_y_orig.shape[0],1))
    test_set_y_orig = test_set_y_orig.reshape((test_set_y_orig.shape[0],1))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [ ]:
# Reshape the training and test examples
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1)
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1)

# Standardize the dataset
train_set_x = train_set_x_flatten/255
test_set_x = test_set_x_flatten/255

In [ ]:
print ("train_set_x shape: " + str(train_set_x.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x  shape: " + str(test_set_x.shape))
print ("test_set_y  shape: " + str(test_set_y.shape))

In [ ]:
# visualize a sample data
index = 13
plt.imshow(train_set_x_orig[index])

In [ ]:
# Create a source dataset from your training data
dataset = tf.data.Dataset.from_tensor_slices((train_set_x,train_set_y))
dataset = dataset.shuffle(buffer_size=64).batch(32)

# Instantiate a simple classification model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(8, activation=tf.nn.relu, dtype='float64'),
  tf.keras.layers.Dense(8, activation=tf.nn.relu, dtype='float64'),
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, dtype='float64')
])

# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.BinaryCrossentropy()

# Instantiate an accuracy metric.
accuracy = tf.keras.metrics.BinaryAccuracy()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

In [ ]:
for i in range(500):    
    # Iterate over the batches of the dataset.
    for step, (x, y) in enumerate(dataset):
      # Open a GradientTape.
      with tf.GradientTape() as tape:

        # Forward pass.
        logits = model(x)

        # Loss value for this batch.
        loss_value = loss(y, logits)
          
      # Get gradients of loss wrt the weights.
      gradients = tape.gradient(loss_value, model.trainable_weights)

      # Update the weights of our linear layer.
      optimizer.apply_gradients(zip(gradients, model.trainable_weights))

      # Update the running accuracy.
      accuracy.update_state(y, logits)

    # Logging.
    if i % 100 == 0:
      print('epoch:', i)
      print('Loss from last step: %.3f' % loss_value)
      print('Total running accuracy so far: %.3f' % accuracy.result())

In [ ]:
# Create a source dataset from your test data
test_dataset = tf.data.Dataset.from_tensor_slices((test_set_x, test_set_y))
test_dataset = test_dataset.batch(32)

# This clears the internal state of the metric
accuracy.reset_states()  

for step, (x, y) in enumerate(test_dataset):
  logits = model(x)
  accuracy.update_state(y, logits)

print('Final test accuracy: %.3f' % accuracy.result())

This exercise consists of the experimentation regarding the Cat vs Non-Cat model previously presented.
<br><br>
Evaluate the performance of the model (train and test) considering a different range of layers (1, 3, and 4) and units dimensions (16,32,64) <br>
Experiment with other optimizers and evaluate different learning rate values (0.001, 0.01, 0.1, 1). What happens?<br>
Has the size of mini-batch a huge influence under results? Make some experiments to support your findings.<br>
Consider this reference and investigate different evaluation scenarios under the hyperparameter search.<br>